## Part 3 - 10/6/25
You have historical data for all ft unit of measurements for GSL height level

GOAL: Compile data through time - say windows of 5 years and do basic statistics


In [32]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import shapely
dir_ = Path("data/usgs_history")  # Use forward slashes for consistency

# Collect all GeoDataFrames in a list
gdfs = []

for file in dir_.iterdir():
    if file.suffix == ".csv":
        print(f"Reading: {file}")
        df = pd.read_csv(file)

        # Ensure lat/lon columns exists
        if 'latitude' in df.columns and 'longitude' in df.columns:
            # Create geometry from lon/lat
            geometry = [shapely.Point(xy) for xy in zip(df['longitude'], df['latitude'])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
        gdfs.append(gdf)
        
if gdfs:
    complete_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
else:
    complete_gdf = gpd.GeoDataFrame()

gsl_gdf = gpd.GeoDataFrame(complete_gdf.drop(columns=["statistic_id", "approval_status", "parameter_code", "last_modified", "time_series_id", "qualifier"]).sort_values("time"))
gsl_gdf.to_csv("data/gsl_wtr_level_history.csv", index=False)
gsl_gdf.to_parquet("data/gsl_wtr_level_history.parquet")
gsl_gdf

Reading: data\usgs_history\GSL_WTR_LEVEL_HISTORY_USGS-10010000.csv
Reading: data\usgs_history\GSL_WTR_LEVEL_HISTORY_USGS-10010024.csv
Reading: data\usgs_history\GSL_WTR_LEVEL_HISTORY_USGS-10010027.csv
Reading: data\usgs_history\GSL_WTR_LEVEL_HISTORY_USGS-10010050.csv
Reading: data\usgs_history\GSL_WTR_LEVEL_HISTORY_USGS-10010100.csv
Reading: data\usgs_history\GSL_WTR_LEVEL_HISTORY_USGS-10141000.csv


,time,monitoring_location_id,value,unit_of_measure,longitude,latitude,geometry
9858,1847-10-18,USGS-10010000,4199.8,ft,-112.213557,40.731333,POINT (-112.21356 40.73133)
13330,1848-07-04,USGS-10010000,4201.0,ft,-112.213557,40.731333,POINT (-112.21356 40.73133)
13627,1848-09-30,USGS-10010000,4199.8,ft,-112.213557,40.731333,POINT (-112.21356 40.73133)
13078,1849-06-16,USGS-10010000,4201.1,ft,-112.213557,40.731333,POINT (-112.21356 40.73133)
2235,1849-10-19,USGS-10010000,4199.9,ft,-112.213557,40.731333,POINT (-112.21356 40.73133)
...,...,...,...,...,...,...,...
29566,2025-10-04,USGS-10010100,4191.0,ft,-112.496903,41.255211,POINT (-112.4969 41.25521)
8815,2025-10-05,USGS-10010000,4191.2,ft,-112.213557,40.731333,POINT (-112.21356 40.73133)
34228,2025-10-05,USGS-10010100,4190.8,ft,-112.496903,41.255211,POINT (-112.4969 41.25521)
16466,2025-10-05,USGS-10010024,4191.1,ft,-112.757206,41.220872,POINT (-112.75721 41.22087)


In [ ]:
# Now Analyze time series data for rolling average - simple start - graph by 5 years

In [70]:
import geopandas as gpd
gdf = gpd.read_parquet("data/gsl_wtr_level_history.parquet")
gdf["YEAR"] = gdf["time"].apply(lambda x: str(x).split("-")[0])
gdf["MONTH"] = gdf["time"].apply(lambda x: str(x).split("-")[1])
gdf = gdf.reset_index(drop=True).sort_values("YEAR")

gdf_grouped = gdf.groupby(["YEAR", "MONTH"]).agg(
    total_value=("value", "sum"),
    item_count=("value", "count"),
    unique_stations=("monitoring_location_id", "nunique")
).reset_index()
gdf_grouped["avg_value"] = gdf_grouped["total_value"] / gdf_grouped["item_count"]
gdf_grouped

,YEAR,MONTH,total_value,item_count,unique_stations,avg_value
0,1847,10,4199.8,1,1,4199.800000
1,1848,07,4201.0,1,1,4201.000000
2,1848,09,4199.8,1,1,4199.800000
3,1849,06,4201.1,1,1,4201.100000
4,1849,10,4199.9,1,1,4199.900000
...,...,...,...,...,...,...
1768,2025,06,377348.3,90,4,4192.758889
1769,2025,07,389873.3,93,4,4192.186022
1770,2025,08,389814.4,93,4,4191.552688
1771,2025,09,377204.0,90,4,4191.155556


In [71]:
for index, row in gdf_grouped.iterrows():
    print(f"YEAR/MONTH: {row['YEAR']}/{row['MONTH']} - {row['avg_value']} - n: {row["item_count"]}")

YEAR/MONTH: 1847/10 - 4199.8 - n: 1
YEAR/MONTH: 1848/07 - 4201.0 - n: 1
YEAR/MONTH: 1848/09 - 4199.8 - n: 1
YEAR/MONTH: 1849/06 - 4201.1 - n: 1
YEAR/MONTH: 1849/10 - 4199.9 - n: 1
YEAR/MONTH: 1850/06 - 4201.2 - n: 1
YEAR/MONTH: 1850/10 - 4199.9 - n: 1
YEAR/MONTH: 1851/07 - 4202.0 - n: 1
YEAR/MONTH: 1851/12 - 4200.6 - n: 1
YEAR/MONTH: 1852/07 - 4202.9 - n: 1
YEAR/MONTH: 1852/11 - 4201.9 - n: 1
YEAR/MONTH: 1853/07 - 4204.0 - n: 1
YEAR/MONTH: 1854/01 - 4203.0 - n: 1
YEAR/MONTH: 1854/07 - 4204.4 - n: 1
YEAR/MONTH: 1854/12 - 4203.6 - n: 1
YEAR/MONTH: 1855/06 - 4204.6 - n: 1
YEAR/MONTH: 1855/10 - 4203.1 - n: 1
YEAR/MONTH: 1856/06 - 4204.3 - n: 1
YEAR/MONTH: 1856/12 - 4203.1 - n: 1
YEAR/MONTH: 1857/07 - 4203.9 - n: 1
YEAR/MONTH: 1857/10 - 4202.1 - n: 1
YEAR/MONTH: 1858/07 - 4203.1 - n: 1
YEAR/MONTH: 1858/11 - 4201.1 - n: 1
YEAR/MONTH: 1859/06 - 4201.8 - n: 1
YEAR/MONTH: 1859/10 - 4200.3 - n: 1
YEAR/MONTH: 1860/07 - 4201.5 - n: 1
YEAR/MONTH: 1860/12 - 4199.5 - n: 1
YEAR/MONTH: 1861/07 - 4201.0